# Image Processing and Normalization
![](2024-08-14-18-03-16.png)
## Author: Diversa
## Project: Volcanes_ML
## Contact: hello@diversa.studio
---

In [1]:
!pip install -U albumentations opencv-python

%cd flirpy
# Install the dependencies
!pip install -r requirements.txt

# Install the library locally
!pip install .


c:\Users\sedig\OneDrive\Escritorio\DIVERSA\VOLCANES\volcanes_ml\scripts\flirpy


C:\Users\sedig\AppData\Roaming\Python\Python312\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Processing c:\users\sedig\onedrive\escritorio\diversa\volcanes\volcanes_ml\scripts\flirpy
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for flirpy: filename=flirpy-0.4.0-py3-none-any.whl size=10188308 sha256=266f7b068f1b0e8c43cec117d2586e20878217fa0c5526c6c01d780eb70e07be
  Stored in directory: C:\Users\sedig\AppData\Local\Temp\pip-ephem-wheel-cache-usc2lfdm\wheels\37\cc\8f\eda4fac8eac05978d3e38b0325993a4b44f1dc09ab63538b2c
Successfully built flirpy
  Attempting uninstall: flirpy
    Found existing installation: flirpy 0.4.0
    Uninstalling flirpy-0.4.0:
      Successfully uninstalled flirpy-0.4.0


In [2]:
!pip uninstall transformers

^C


In [ ]:
%pip install transformers==4.12.5


In [ ]:
%pip install matplotlib
%pip install pandas
%pip install numpy
import matplotlib.pyplot as plt
import pandas as pd
from flirpy.io.fff import Fff
import numpy as np
import  os
import torch
from transformers import SegFormerFeatureExtractor, SegFormerForSemanticSegmentation

In [6]:
root_dir = "C:/Users/sedig/OneDrive/Escritorio/DIVERSA/VOLCANES/volcanes_ml/Cotopaxi"

# Define the output root directory for PNG files
output_root_dir = 'C:/Users/sedig/OneDrive/Escritorio/DIVERSA/VOLCANES/volcanes_ml/out_norm_png'

# Walk through the directory tree
for subdir, dirs, files in os.walk(root_dir):
    for file in files:
        if file.endswith('.fff'):
            file_path = os.path.join(subdir, file)

            try:
                # Load an FFF file
                fff_reader = Fff(file_path)

                # Read the thermal image
                thermal_image = fff_reader.get_image()

                # Normalize the thermal image
                thermal_image_normalized = (thermal_image - thermal_image.min()) / (thermal_image.max() - thermal_image.min())

                # Create the corresponding output directory structure
                relative_path = os.path.relpath(subdir, root_dir)
                output_dir = os.path.join(output_root_dir, relative_path + '_png')
                os.makedirs(output_dir, exist_ok=True)

                # Save the PNG file in the output directory
                output_file_path = os.path.join(output_dir, f"{os.path.splitext(file)[0]}.png")
                plt.imshow(thermal_image_normalized, cmap='inferno')
                plt.axis('off')  # Remove the axes
                plt.savefig(output_file_path, bbox_inches='tight', pad_inches=0)
                plt.close()

            except Exception as e:
                print(f"Error processing {file_path}: {e}")


In [ ]:
feature_extractor = SegFormerFeatureExtractor.from_pretrained('nvidia/segformer-b0-finetuned-airbus')
model = SegFormerForSemanticSegmentation.from_pretrained('nvidia/segformer-b0-finetuned-airbus')

def process_image(image_path, output_dir):
    # Load the image
    image = Image.open(image_path).convert("RGB")  # Ensure it's in RGB mode

    # Preprocess the image
    inputs = feature_extractor(images=image, return_tensors="pt")

    # Perform segmentation
    outputs = model(**inputs)
    logits = outputs.logits
    segmentation = torch.argmax(logits, dim=1)[0]

    # Convert segmentation to a numpy array
    segmentation = segmentation.numpy()

    # Assuming you know the label IDs for sky and mountain/volcano
    sky_label = 12  # Example label ID for sky
    mountain_label = 21  # Example label ID for mountain/volcano

    # Create masks
    sky_mask = (segmentation == sky_label)
    mountain_mask = (segmentation == mountain_label)

    # Apply the masks to the original image
    sky_image = np.array(image) * np.expand_dims(sky_mask, axis=-1)
    mountain_image = np.array(image) * np.expand_dims(mountain_mask, axis=-1)

    # Combine masks for the overlay
    combined_mask = sky_mask | mountain_mask
    combined_image = np.array(image) * np.expand_dims(combined_mask, axis=-1)

    # Convert back to Image for saving
    sky_image = Image.fromarray(sky_image.astype(np.uint8))
    mountain_image = Image.fromarray(mountain_image.astype(np.uint8))
    combined_image = Image.fromarray(combined_image.astype(np.uint8))

    # Define output paths
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    sky_output_path = os.path.join(output_dir, f"{base_name}_sky.png")
    mountain_output_path = os.path.join(output_dir, f"{base_name}_mountain.png")
    combined_output_path = os.path.join(output_dir, f"{base_name}_combined.png")

    # Save the segmented images
    sky_image.save(sky_output_path)
    mountain_image.save(mountain_output_path)
    combined_image.save(combined_output_path)

    print(f"Processed {image_path}: saved sky to {sky_output_path}, mountain to {mountain_output_path}, and combined to {combined_output_path}")

In [ ]:
def process_directory(root_dir, output_dir)
    os.makedirs(output_dir, exist_ok=True)
    
    for subdir, _, files in os.walk(root_dir):
        for file in files:
            if file.endswith('.png'):
                file_path = os.path.join(subdir, file)
                process_image(file_path, output_dir)

In [ ]:
root_dir = ''
output_dir = ''
process_directory(root_dir, output_dir)